### 密钥创建类

正在测试节点名称为1是否好用  ok好用

下一步是测试生成的节点是否可用 ok 好用

使我的主机加入网络试试 unreachable 的  可用中转

### 报错:
-1 网络号未使用私有地址

-2 目前仅支持A,B,C类地址

-3 当前网络不支持这么多主机

In [1]:
import os
import uuid
import shutil
import ipaddress
class nodes_builder():
    totleNodes = 0
    confDir = ""
    uuid = ""
    network = ""
    netmask = ""
    def __init__(self,num,network="10.1.0.0/24"):
        self.totleNodes = num
        self.network = ipaddress.IPv4Network(network)
        if not self.network.is_private:
            print("error -1")
        if not(self.network.netmask.exploded == '255.255.255.0' or
            self.network.netmask.exploded == '255.255.0.0' or
            self.network.netmask.exploded == '255.0.0.0'):
            print("error -2")
        if num > self.network.num_addresses-2:
            print("error -3")
        self.netmask = self.network.netmask.exploded
    def BuildUuid6(self):
        self.uuid = uuid.uuid4().hex[:6]
    def BuildConfigDir(self):
        if not os.path.exists('./configs'):
            os.mkdir('./configs')
        self.confDir = './configs/config-'+self.uuid+'/'
        if not os.path.exists(self.confDir):
            os.mkdir(self.confDir)
            for i in range(1,self.totleNodes+1):
                os.mkdir(self.confDir+str(i))
                os.mkdir(self.confDir+str(i)+'/'+'security')
                os.mkdir(self.confDir+str(i)+'/'+'ed25519')
            return True
        else:
            return False
    def BuildSecurity(self):
        for i in range(1,self.totleNodes+1):
            pathToPrivateKey = self.confDir + str(i) + "/security/" + str(i)+ ".private"
            pathToPublicKey = self.confDir + str(i) + "/security/" + str(i)+ ".public"
            os.system("./gnb_crypto -c -p "+pathToPrivateKey+" -k "+pathToPublicKey)
    def BuildEd25519(self):
        for i in range(1,self.totleNodes+1):
            pathToPublicKey = self.confDir + str(i) + "/security/" + str(i)+ ".public"
            for i in range(1,self.totleNodes+1):
                pathToEd25519 = self.confDir + str(i) + "/ed25519"
                os.system("cp "+pathToPublicKey+" "+pathToEd25519)
    def BuildAddressConf(self):
        for i in range(1,self.totleNodes+1):
            pathToNode = self.confDir + str(i)
            os.system("cp ./defaultConf/address.conf "+ pathToNode)
    def BuildNodeConf(self):
        for i in range(1,self.totleNodes+1):
            pathToNode = self.confDir + str(i)
            os.system("cp ./defaultConf/node.conf "+ pathToNode)
            os.system("sed -i 's/\$nodeid/"+str(i)+"/g' "+pathToNode+"/node.conf")
    def BuildRouteConf(self):
        route = ""
        for i in range(1,self.totleNodes+1):
            route+=str(i)+'|'+(self.network.network_address+i).exploded+'|'+self.netmask+'\n'
        print(route)
        for i in range(1,self.totleNodes+1):
            with open(self.confDir+str(i)+'/route.conf','w') as f:
                f.write(route)
                f.close()
    def BuildBinary(self):
        for i in range(1,self.totleNodes+1):
            os.system("cp -r ./bin/ "+self.confDir+str(i))
        
    def BuildAll(self):
        self.BuildUuid6()
        self.BuildConfigDir()
        self.BuildSecurity()
        self.BuildEd25519()
        self.BuildAddressConf()
        self.BuildNodeConf()
        self.BuildRouteConf()
        self.BuildBinary()
            
    def CleanConf(self):
        if os.path.exists('./configs'):
            shutil.rmtree('./configs')
        


In [4]:
a = nodes_builder(7)


In [5]:
a.BuildAll()

1|10.1.0.1|255.255.255.0
2|10.1.0.2|255.255.255.0
3|10.1.0.3|255.255.255.0
4|10.1.0.4|255.255.255.0
5|10.1.0.5|255.255.255.0
6|10.1.0.6|255.255.255.0
7|10.1.0.7|255.255.255.0



In [5]:
a.CleanConf()

In [3]:
os.popen("pwd").read()[:-1]

'/home/esp3j0/code/python/p2pvpn'